In [1]:
import numpy as np
import pandas as pd

In [2]:
patent_abstract = pd.read_csv('US_patent_abstract_5000_2015_with_title_1.csv')
patent_abstract.shape

(5000, 30)

In [3]:
patent_abstract['quality_rank'].value_counts()

0    2896
1    2104
Name: quality_rank, dtype: int64

In [4]:
2896/5000

0.5792

In [5]:
data_train = patent_abstract

In [6]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [7]:
MAX_SENT_LENGTH = 30
MAX_SENTS = 30
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [8]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [9]:
import nltk
from nltk import tokenize

claims = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.text.shape[0]): # for each row
    text = clean_str(str(data_train.iloc[idx]['claims_text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence 
    claims.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data)
    labels.append(data_train.iloc[idx]['quality_rank']) # label is just a list holding our label which is quality_index

In [10]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [11]:
tokenizer.word_counts

OrderedDict([('1', 51002),
             ('a', 242716),
             ('device', 25594),
             ('comprising', 30079),
             ('memory', 5885),
             ('to', 138153),
             ('store', 1110),
             ('instructions', 2402),
             ('where', 2596),
             ('the', 502964),
             ('is', 74005),
             ('first', 61679),
             ('and', 130429),
             ('processor', 4393),
             ('execute', 428),
             ('receive', 2862),
             ('message', 3117),
             ('second', 53240),
             ('from', 32429),
             ('differ', 24),
             ('includes', 7905),
             ('header', 425),
             ('event', 1127),
             ('data', 20784),
             ('identify', 582),
             ('based', 9046),
             ('on', 25291),
             ('that', 24349),
             ('messages', 457),
             ('are', 14702),
             ('problem', 62),
             ('reports', 145),
             ('p

In [12]:
tokenizer.document_count

5000

In [13]:
len(tokenizer.word_index) # 143937 unqiue vocabulary out of the entire abstract dataset

31722

In [15]:
for i, sentences in enumerate(claims):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
            #for word in wordTokens:
                if word in tokenizer.word_index.keys():
                    if (k < MAX_SENT_LENGTH) and (tokenizer.word_index[word] < MAX_NB_WORDS):
                        data[i, j, k] = tokenizer.word_index[word]
                        k = k + 1

In [16]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 31722 unique tokens.


In [17]:
np.unique(np.asarray(labels))

array([0, 1])

In [18]:
labels = to_categorical(np.asarray(labels), num_classes=2) 
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 30, 30)
Shape of label tensor: (5000, 2)


In [19]:
#indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
#data = data[indices]
#labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [20]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[2320. 1680.]
[576. 424.]


In [112]:
x_train[2]

array([[ 320,    7,   82,    7,   13,   14,   56,  150,   53,   21,   14,
          56,  150,   44,   43,    4,  180,   14,   56,   27,    2, 1643,
          58,   28,    7, 2179,   25,   57, 1120,   25],
       [  20,    1,   56,  150,   53,   31,    4,    8,   13,    6,    1,
          52,   44,    7,   43,    4,  497,    1,   56,  150,   44,    4,
         180,   14,   56,   23,  919,   57, 1515,   59],
       [  29,    1,   56,  150,   53,   31,    4,    8,   13,    6,    2,
         291,    3,    1,  919, 6605,    7,  961,  261,   70,   18,  367,
           4,    2,  464,    3,    2,   62,  259,  140],
       [  32,    1,   56,  150,   53,   31,    4,    8,   13,   37,   21,
          14, 1147,   44,   43,    4, 3895,    1, 1643,   58,    6,    1,
          52,   44,    7,   43,    4,  497,    1, 1147],
       [  41,    1,   56,  150,   53,   31,    4,    8,   13,    6,    1,
          52,   44,    7,   43,    4,  497,    1,   56,  150,   44,    4,
         180,   14,   56,   23, 

In [113]:
x_val[10]

array([[  13,    2,  918,  136,   21,   14, 1400,  145,    5,   14, 1028,
        1276,  442,    4,    1, 1400,  145,    5,  684,    3, 1197,   79,
           5,    1, 1028, 1276,   21,   15,   24,   96],
       [  20,    1,  918,  136,   38, 1821,    9,    8,   13,    6,    2,
          33,    3, 1493,   42,  120,  229,   14,  899,   93,    3,    1,
        1028, 1276,    5, 4509,    1, 1028, 1276,  101],
       [  29,    1,  918,  136,   38, 1821,    9,    8,   13,    6,   16,
          60,    3,    1, 1028, 1276,    7,  442,    4,    1, 1400,  145,
           5,    1,  126,   60,    3,    1, 1028, 1276],
       [  32,    1,  918,  136,   38, 1821,    9,    8,   13,    6,    1,
         849,  160,  156,   39,    1,  686,  160,  156,   39,    5,    1,
        3568,  156,   39,   42,    3,  367,  391, 1027],
       [  41,    1,  918,  136,   38, 1821,    9,    8,   32,    6,    1,
         689,    7, 1894,    9,    1, 2343,  106,    0,    0,    0,    0,
           0,    0,    0,    0, 

In [114]:
x_train.shape

(4000, 20, 30)

In [21]:
np.unique(y_val, return_counts=True)

(array([0., 1.], dtype=float32), array([1000, 1000]))

In [22]:
# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display

# NumPy and SciPy for matrix ops

import scipy.sparse


# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import glove_helper; reload(glove_helper)

<module 'glove_helper' from '/home/nobu_yamaguchi/final_project/glove_helper.py'>

In [23]:
GLOVE_DIR = "data/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [24]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [25]:

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

(31723, 100)


In [26]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=False)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

claims_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
claims_encoder = TimeDistributed(sentEncoder)(claims_input)
l_lstm_sent = Bidirectional(LSTM(100))(claims_encoder)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_sent)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_dense)
l_lstm_sent_drop = Dropout(rate=0.3)(l_lstm_dense)
preds = Dense(2, activation='sigmoid')(l_lstm_sent_drop) # [Steven] why 5 will throw error and require 6 ? it is weird since we only have 5 number scale
model = Model(claims_input, preds)

In [27]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [28]:
print("model fitting - Hierachical LSTM max_sent = 30")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM max_sent = 20
Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 65s 16ms/step - loss: 0.6322 - acc: 0.6472 - val_loss: 0.6419 - val_acc: 0.6265
Epoch 2/10
4000/4000 [==============================] - 59s 15ms/step - loss: 0.6216 - acc: 0.6536 - val_loss: 0.6271 - val_acc: 0.6500
Epoch 3/10
4000/4000 [==============================] - 59s 15ms/step - loss: 0.6125 - acc: 0.6578 - val_loss: 0.6268 - val_acc: 0.6665
Epoch 4/10
4000/4000 [==============================] - 59s 15ms/step - loss: 0.6003 - acc: 0.6741 - val_loss: 0.6403 - val_acc: 0.6290
Epoch 5/10
4000/4000 [==============================] - 59s 15ms/step - loss: 0.5920 - acc: 0.6824 - val_loss: 0.6325 - val_acc: 0.6075
Epoch 6/10
4000/4000 [==============================] - 59s 15ms/step - loss: 0.5874 - acc: 0.6927 - val_loss: 0.6403 - val_acc: 0.6620
Epoch 7/10
4000/4000 [==============================] - 59s 15ms/step - loss: 0.5747 - ac

In [30]:
print("model fitting - Hierachical LSTM max_sent = 20")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM
Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 48s 12ms/step - loss: 0.6436 - acc: 0.6339 - val_loss: 0.6200 - val_acc: 0.6650
Epoch 2/10
4000/4000 [==============================] - 44s 11ms/step - loss: 0.6385 - acc: 0.6423 - val_loss: 0.6220 - val_acc: 0.6605
Epoch 3/10
4000/4000 [==============================] - 43s 11ms/step - loss: 0.6319 - acc: 0.6449 - val_loss: 0.6109 - val_acc: 0.6665
Epoch 4/10
4000/4000 [==============================] - 43s 11ms/step - loss: 0.6245 - acc: 0.6551 - val_loss: 0.6078 - val_acc: 0.6640
Epoch 5/10
4000/4000 [==============================] - 43s 11ms/step - loss: 0.6170 - acc: 0.6559 - val_loss: 0.6103 - val_acc: 0.6580
Epoch 6/10
4000/4000 [==============================] - 43s 11ms/step - loss: 0.6041 - acc: 0.6694 - val_loss: 0.6272 - val_acc: 0.6290
Epoch 7/10
4000/4000 [==============================] - 43s 11ms/step - loss: 0.5925 - acc: 0.6916 - va

In [99]:
print("model fitting - Hierachical LSTM Max_sent = 15")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM Max_sent = 15
Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 35s 9ms/step - loss: 0.6447 - acc: 0.6219 - val_loss: 0.6432 - val_acc: 0.6145
Epoch 2/10
4000/4000 [==============================] - 32s 8ms/step - loss: 0.6381 - acc: 0.6306 - val_loss: 0.6430 - val_acc: 0.6220
Epoch 3/10
4000/4000 [==============================] - 33s 8ms/step - loss: 0.6307 - acc: 0.6403 - val_loss: 0.6466 - val_acc: 0.6085
Epoch 4/10
4000/4000 [==============================] - 32s 8ms/step - loss: 0.6244 - acc: 0.6478 - val_loss: 0.6361 - val_acc: 0.6335
Epoch 5/10
4000/4000 [==============================] - 32s 8ms/step - loss: 0.6098 - acc: 0.6634 - val_loss: 0.6480 - val_acc: 0.6335
Epoch 6/10
4000/4000 [==============================] - 33s 8ms/step - loss: 0.6009 - acc: 0.6780 - val_loss: 0.6372 - val_acc: 0.6380
Epoch 7/10
4000/4000 [==============================] - 32s 8ms/step - loss: 0.5853 - acc: 0.69

In [76]:
print("model fitting - Hierachical LSTM Max sentence = 10")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM Max sentence = 10
Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 26s 7ms/step - loss: 0.6543 - acc: 0.5889 - val_loss: 0.6533 - val_acc: 0.5825
Epoch 2/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.6454 - acc: 0.6140 - val_loss: 0.6506 - val_acc: 0.6100
Epoch 3/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.6333 - acc: 0.6295 - val_loss: 0.6492 - val_acc: 0.6010
Epoch 4/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.6276 - acc: 0.6392 - val_loss: 0.6587 - val_acc: 0.5775
Epoch 5/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.6133 - acc: 0.6480 - val_loss: 0.6711 - val_acc: 0.5970
Epoch 6/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.6010 - acc: 0.6654 - val_loss: 0.6698 - val_acc: 0.5930
Epoch 7/10
4000/4000 [==============================] - 24s 6ms/step - loss: 0.5780 - acc: 